In [6]:
import pandas as pd
from pyedflib.highlevel import read_edf_header
import os
import re

In [2]:
DATA_PATH = ".\\data\\CHB"

In [3]:
patient_info = pd.read_csv("patient_info_chb_origin.csv")

In [5]:
patient_info.head()

,name,start,end,state
0,CHB001,3371856174,3371859785,interictal
1,CHB001,3371859785,3371861465,preictal_early
2,CHB001,3371861465,3371863265,preictal_ontime
3,CHB001,3371863265,3371863385,preictal_late
4,CHB001,3371863385,3371866380,ictal


In [9]:
chb_dir_list = [dir_name for dir_name in os.listdir(DATA_PATH) if re.match("CHB[0-9]{3}", dir_name)]
for chb_dir in chb_dir_list:
    edf_dir_path = os.path.join(DATA_PATH, chb_dir)
    edf_list = [file_name for file_name in os.listdir(edf_dir_path) if file_name.endswith('.edf')]
    
    for edf in edf_list:
        
    

['CHB001_01.edf', 'CHB001_02.edf', 'CHB001_03.edf', 'CHB001_04.edf', 'CHB001_05.edf', 'CHB001_06.edf', 'CHB001_07.edf', 'CHB001_08.edf', 'CHB001_09.edf', 'CHB001_10.edf', 'CHB001_11.edf', 'CHB001_12.edf', 'CHB001_13.edf', 'CHB001_14.edf', 'CHB001_15.edf', 'CHB001_16.edf', 'CHB001_17.edf', 'CHB001_18.edf', 'CHB001_19.edf', 'CHB001_20.edf', 'CHB001_21.edf', 'CHB001_22.edf', 'CHB001_23.edf', 'CHB001_24.edf', 'CHB001_25.edf', 'CHB001_26.edf', 'CHB001_27.edf', 'CHB001_29.edf', 'CHB001_30.edf', 'CHB001_31.edf', 'CHB001_32.edf', 'CHB001_33.edf', 'CHB001_34.edf', 'CHB001_36.edf', 'CHB001_37.edf', 'CHB001_38.edf', 'CHB001_39.edf', 'CHB001_40.edf', 'CHB001_41.edf', 'CHB001_42.edf', 'CHB001_43.edf', 'CHB001_46.edf']
['CHB002_01.edf', 'CHB002_02.edf', 'CHB002_03.edf', 'CHB002_04.edf', 'CHB002_05.edf', 'CHB002_06.edf', 'CHB002_07.edf', 'CHB002_08.edf', 'CHB002_09.edf', 'CHB002_10.edf', 'CHB002_11.edf', 'CHB002_12.edf', 'CHB002_13.edf', 'CHB002_14.edf', 'CHB002_15.edf', 'CHB002_16+.edf', 'CHB002_16.